# [585. Investments in 2016](https://leetcode.com/problems/investments-in-2016/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Insurance

<pre>+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| pid         | int   |
| tiv_2015    | float |
| tiv_2016    | float |
| lat         | float |
| lon         | float |
+-------------+-------+</pre>
pid is the primary key (column with unique values) for this table.
Each row of this table contains information about one policy where:
pid is the policyholder's policy ID.
tiv_2015 is the total investment value in 2015 and tiv_2016 is the total investment value in 2016.
lat is the latitude of the policy holder's city. It's guaranteed that lat is not NULL.
lon is the longitude of the policy holder's city. It's guaranteed that lon is not NULL.
 

Write a solution to report the sum of all total investment values in 2016 tiv_2016, for all policyholders who:

have the same tiv_2015 value as one or more other policyholders, and
are not located in the same city as any other policyholder (i.e., the (lat, lon) attribute pairs must be unique).
Round tiv_2016 to two decimal places.

The result format is in the following example.

Example 1:

Input: 
Insurance table:
<pre>+-----+----------+----------+-----+-----+
| pid | tiv_2015 | tiv_2016 | lat | lon |
+-----+----------+----------+-----+-----+
| 1   | 10       | 5        | 10  | 10  |
| 2   | 20       | 20       | 20  | 20  |
| 3   | 10       | 30       | 20  | 20  |
| 4   | 10       | 40       | 40  | 40  |
+-----+----------+----------+-----+-----+</pre>
Output: 
<pre>+----------+
| tiv_2016 |
+----------+
| 45.00    |
+----------+</pre>
Explanation: 
The first record in the table, like the last record, meets both of the two criteria.
The tiv_2015 value 10 is the same as the third and fourth records, and its location is unique.

The second record does not meet any of the two criteria. Its tiv_2015 is not like any other policyholders and its location is the same as the third record, which makes the third record fail, too.
So, the result is the sum of tiv_2016 of the first and last record, which is 45.

In [1]:
# pandas schema

import pandas as pd

data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype(
    {'pid': 'Int64', 'tiv_2015': 'Float64', 'tiv_2016': 'Float64', 'lat': 'Float64', 'lon': 'Float64'})

In [2]:
# pandas df to pyspark df

import pyspark.sql.functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

insurance_df = spark.createDataFrame(insurance)
insurance_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/06 16:09:47 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/06 16:09:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/06 16:09:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+--------+--------+----+----+
|pid|tiv_2015|tiv_2016|lat |lon |
+---+--------+--------+----+----+
|1  |10.0    |5.0     |10.0|10.0|
|2  |20.0    |20.0    |20.0|20.0|
|3  |10.0    |30.0    |20.0|20.0|
|4  |10.0    |40.0    |40.0|40.0|
+---+--------+--------+----+----+


In [3]:
from pyspark.sql.window import Window

# in Spark Dataframe

insurance_df.withColumn('filter_location', F.count(F.concat(F.col('lat'), F.col('lon')))
                        .over(Window.partitionBy(F.concat(F.col('lat'), F.col('lon'))))). \
    withColumn('filter_location', F.col('filter_location') == 1). \
    withColumn('filter_tiv_2015', F.count('tiv_2015').over(Window.partitionBy('tiv_2015'))). \
    withColumn('filter_tiv_2015', F.col('filter_tiv_2015') > 1). \
    filter((F.col('filter_tiv_2015') == True) & (F.col('filter_location') == True)). \
    agg(F.round(F.sum('tiv_2016'), 2).alias('tiv_2016')).show()

+--------+
|tiv_2016|
+--------+
|    45.0|
+--------+


In [4]:
# in Spark SQL

insurance_df.createOrReplaceTempView('insurance')
spark.sql('''
select round(sum(tiv_2016),2) as tiv_2016 from 
(
select 
    *,
    count(tiv_2015) over (partition by tiv_2015) as filter_tiv_2015,
    count(concat(lat,lon)) over (partition by concat(lat,lon)) as filter_location
from insurance
) i where filter_tiv_2015>1 and filter_location=1
''').show()

+--------+
|tiv_2016|
+--------+
|    45.0|
+--------+


In [5]:
spark.stop()